# DapticsClient - Introduction

This notebook contains an interactive introduction to the Python DapticsClient class,
a simplified interface for accessing the Daptics GraphQL API for the optimization of
experimental design.

Documentation for using the DapticsClient class (implemented in the daptics_client.py
file in this folder) is included as comment lines in the interactive Python cells of
this notebook.

For additional help or information, please visit or contact Daptics.

On the web at https://daptics.ai
By email at support@daptics.ai

Daptics API Version 0.10.0
Copyright (c) 2019 Daptics Inc.

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), the rights to use, copy, modify, merge, publish, and/or distribute, copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

You do not have the right to sub-license or sell copies of the Software.

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

In [ ]:
# Step 1. Create a DapticsClient object.

# Before running this project, please make sure that your Jupyter
# Python environment has these required packages installed:
#
#   chardet
#   urllib3
#   requests
#   gql
#
# You will also need a validated user account on the Daptics API server.
# You can create an account by clicking the `Get Started for Free` button at
# https://daptics.ai
#
# See the 01_README.ipynb notebook in this folder for more information.

# Import classes from the daptics_client module
# Requirements are Python 3, and the `gql` and `requests` libraries
from daptics_client import DapticsClient, DapticsTaskType, DapticsExperimentsType
from datetime import datetime

# Create a client object and try to connect to the daptics API server.
# The client constructor takes a URL argument that specifies the scheme,
# host and port number of the API server.

# Note: do not specify the path on the server; the DapticsClient object
# will take care of that for us.
api_host = 'https://api.daptics.ai'
daptics = DapticsClient(api_host)

# Show the options set in the client.
# The default options leave everything under the user's manual control.
print(daptics.options)

In [ ]:
# Step 2. Connect to the API server and show GraphQL introspection information.

# The `connect` method will attempt to connect to the /api path on the
# API server and obtain the GraphQL schema.
daptics.connect()

# Show the result. After connecting, the daptics object should have a `gql` attribute.
# Let's look at the data stored in the `gql` attribute by printing its `__dict__`.
# We can see that the gql library has introspected all the GraphQL type,
# query and mutation information exposed by the API.
print(daptics.gql.__dict__)

In [ ]:
# Step 3. Log in to the API server to obtain an access token.

# The `login` method takes two string arguments, the user's
# `email` and `password`.

# Note: Use the real email address and password you used when you created
# your account on the daptics.ai website.

email = 'YOUR_EMAIL@YOUR_DOMAIN'
password = 'YOUR_PASSWORD'
data = daptics.login(email, password)

# Show the result. The `daptics` object will remember the access token
# and use it for all subsequent API requests. The `daptics` object
# will also remember the `userId` that corresponds to the email address.
print(data)

In [ ]:
# Step 4. Create a daptics session on the server.

name = datetime.now().strftime('Analytics Session %Y%m%d-%H%M%S')
description = 'Demonstrate analytics files'
data = daptics.create_session(name, description)

# Show the result. The `daptics` object will remember the `sessionId`.
print(data)

In [ ]:
# Step 5. Save and validate the default experimental space parameters.

# Make a params input from the default `initial_params` for the session.
params = {
    'space': {
        'type': 'factorial'
    },
    'populationSize': 30,
    'replicates': 2
}
fname = 'esd-factorial-4x8.csv'

data = daptics.put_experimental_parameters_csv(fname, params)

# Show the result: a task with 'type' 'space'.
print(data)

In [ ]:
# Step 6. Wait for the experimental space parameters validation result.

data, errors = daptics.wait_for_current_task(DapticsTaskType.SPACE)

# Show the result.
print(data)

In [ ]:
# Step 7. Generate the first design (without any initial experiments).

data = daptics.generate_design()

# Show the result: a task with 'type' 'generate'.
print(data)

In [ ]:
# Step 8. Wait for the first design generation result.

data, errors = daptics.wait_for_current_task(DapticsTaskType.GENERATE)
print(data)

In [ ]:
# Step 9. Fetch the design that was created.

experiments_table = daptics.design['table']
print(experiments_table)

In [ ]:
# Step 10. Simulate some responses and submit them for the next design generation.

data = daptics.simulate_experiment_responses(experiments_table)
print(data)

experiments = data['simulateResponses']['table']
data = daptics.put_experiments(DapticsExperimentsType.DESIGNED_WITH_OPTIONAL_EXTRAS, experiments)
print(data)

In [ ]:
# Step 11. Wait for the responses to be incorporated.

data, errors = daptics.wait_for_current_task(DapticsTaskType.UPDATE)
print(data)

In [ ]:
# Step 12. Generate analytics for the latest generation available.

# The `generate_analytics` method starts an "analytics" task.
data = daptics.generate_analytics()
print(data)

In [ ]:
# Step 13. Wait for the analytics files to be ready.

data, errors = daptics.wait_for_current_task(DapticsTaskType.ANALYTICS)
print(data)

# When the "analytics" task has completed, the data is also stored in the client's
# `analytics` attribute.
print(daptics.analytics)

In [ ]:
# Step 14. Download one of the analytics files.

# The `download_analytics_file` method takes two arguments:
#    `url` - The URL returned from the "analtyics" task result.
#    `fname` - A file path to write the file to.
#
# We will extract the URL and filename from the `files` variable
# we set in the previous step.
files = daptics.analytics['files']
file = files[0]
url = file['url']
fname = file['filename']
data = daptics.download_analytics_file(url, fname)

# Show the result, a response object.
print(data)

# Show the length of the file downloaded.
import os
size = os.path.getsize(fname)
print('Size of {0} is {1} bytes'.format(fname, size))

In [ ]:
# Step 15. Display the PDF in an iframe.

# NOTE: If this shows a black box, try opening the file in a
# PDF viewing application, rather than in the notebook.
# The latest Chrome browsers impose strict security on PDF plugins.

# We'll use a helper class that exposes a `_repr_html_` method.
# We use "embed" tag because Chrome will not let us show an iframe.
class PDF(object):
  def __init__(self, path, height=400):
    self.path = path
    self.height = height

  def _repr_html_(self):
    return '<embed type="application/pdf" src="{0}" width="100%" />'.format(self.path)

  def _repr_latex_(self):
    return r'\includegraphics[width=1.0\textwidth]{{{0}}}'.format(self.path)

# Initialize a PDF object, and let Jupyter notebook show us the iframe representation.
PDF(fname)